In [43]:
import logging
import sisl
import pandas as pd
import csv
import ast
import numpy as np

from pathlib import Path
from tqdm import tqdm

def flatten(xss):
    return [x for xs in xss for x in xs]

def write_structures_paths(structures_paths, filepath):
    with open(filepath, "w") as f:
        f.write("\n".join(structures_paths))

def read_structures_paths(filepath):
    with open(filepath, "r") as f:
        return f.read().splitlines()
    
logging.getLogger().setLevel(logging.WARNING)


In [74]:
# Use only determined subset of the dataset
exclude_carbons=True
use_only= ["8_ATOMS"]
true_dataset_folder = Path("../dataset")
pointers_folder = Path('../dataset_nocarbon')
if use_only is not None:
    x_atoms_paths = [pointers_folder / f"SHARE_OUTPUTS_{n}" for n in use_only]

print(x_atoms_paths)

paths = []

# If you want no carbons
if exclude_carbons:
    # Get all the structures
    filepath = "structures.txt"
    structures_paths = [[] for _ in x_atoms_paths]
    for i, x_atoms_path in enumerate(x_atoms_paths):
        structures = read_structures_paths(str(x_atoms_path / filepath))
        for structure in structures:
            structures_paths[i].append(x_atoms_path.parts[-1] +"/"+ structure)

    # Now we join them with the true parent folder
    for structures in structures_paths:
        for structure in structures:
            true_path = true_dataset_folder / structure
            paths.append(true_path)

    # If you want carbons
else:  
    # Join all structures in the paths variable
    x_atoms_paths = [true_dataset_folder / f"SHARE_OUTPUTS_{n}" for n in use_only]
    for n_atoms_path in x_atoms_paths:
        structure_paths = list(n_atoms_path.glob('*/')) 
        for structure_path in structure_paths:
            paths.append(structure_path)

print(paths)

[PosixPath('../dataset_nocarbon/SHARE_OUTPUTS_8_ATOMS')]
[PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/82b3-4886-4447-b723-43a3ec6aa15c'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/d8c7-b0b4-4d41-895a-c014cdbd8416'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/cca5-7a6f-4d1a-9d8d-9db99d3b9ad9'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/807c-f894-4054-8ea4-da19dab86b43'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/e3fd-ba43-41c3-b53f-997c4a89dbfb'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/b05a-2bc5-4f47-8c55-03787b943376'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/e65d-4965-49b8-b158-22b21b6b6000'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/d972-1e26-4112-a6f0-7e806bc1c05c'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/f980-0604-4608-b493-c98c4b11a3b8'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/b10c-59df-431c-ae6c-b40c0eee9cb9'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/7f49-aa43-4b92-a2b2-a3bf11e36f26'), PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/206b-84a4-448e-9bf

In [75]:
# Plot all structures
found = False
for path in tqdm(paths):
    file = sisl.get_sile(path / "aiida.fdf")
    geom = file.read_geometry()

    import numpy as np

    dec = 1

    x_coords = geom.xyz[:, 0]
    x_coords = np.round(x_coords, dec)

    y_coords = geom.xyz[:, 1]
    y_coords = np.round(y_coords, dec)

    z_coords = geom.xyz[:, 2]
    z_coords = np.round(z_coords, dec)
    
    # Check if there is an atom in plane ZX which has any other atom above him
    y_min = min(y_coords)
    
    for i, y in enumerate(y_coords):
        if y == y_min:
            # Loop over layer 2 atoms
            for j, y1 in enumerate(y_coords):
                if y1 > y_min:
                    # Check if they are in the same z coord
                    if z_coords[j] == z_coords[i]:
                        # Check if they are in the same x coord
                        if x_coords[j] == x_coords[i]:
                            found = True

                            print(path)
                            print("x=", x_coords)
                            print("y=", y_coords)
                            print("z=", z_coords)

                            first_coords = (x_coords[i], y_coords[i], z_coords[i])
                            second_coords = (x_coords[j], y_coords[j], z_coords[j])
                            print(f"\nFound atom {j} in second layer above atom {i} in first layer.")
                            print(f"Coords of the first layer atom:", first_coords)
                            print(f"Coords of the second layer atom:", second_coords)
                            file.plot.geometry(axes="xyz").show()

if not found:
    print("\nNo atoms above first layer atoms in ANY path.")
    



100%|██████████| 399/399 [00:06<00:00, 59.03it/s]


No atoms above first layer atoms in ANY path.


In [77]:
from tqdm import tqdm
import numpy as np
import sisl

found = False

count = 0
paths_sus = []
for path in tqdm(paths):
    file = sisl.get_sile(path / "aiida.fdf")
    geom = file.read_geometry()

    dec = 1  # Decimal precision

    x_coords = np.round(geom.xyz[:, 0], dec)
    y_coords = np.round(geom.xyz[:, 1], dec)
    z_coords = np.round(geom.xyz[:, 2], dec)

    # Check if there's an atom in XY plane with another atom directly above it in Z
    z_min = min(z_coords)

    for i, z in enumerate(z_coords):
        if z == z_min:
            for j, z1 in enumerate(z_coords):
                if z1 > z_min:
                    if x_coords[j] == x_coords[i] and y_coords[j] == y_coords[i]:
                        found = True

                        # print(path)
                        # print("x=", x_coords)
                        # print("y=", y_coords)
                        # print("z=", z_coords)

                        first_coords = (x_coords[i], y_coords[i], z_coords[i])
                        second_coords = (x_coords[j], y_coords[j], z_coords[j])
                        # print(f"\nFound atom {j} in second layer above atom {i} in first layer.")
                        # print("Coords of the first layer atom:", first_coords)
                        # print("Coords of the second layer atom:", second_coords)
                        # file.plot.geometry(axes="xyz").show()
                        paths_sus.append(path)

if not found:
    print("\nNo atoms stacked along Z direction at same XY position in ANY path.")
else:
    print("Found sus.")

100%|██████████| 399/399 [00:06<00:00, 59.17it/s]

Found sus.


In [83]:
i = 20
path = paths_sus[i]

file = sisl.get_sile(path / "aiida.fdf")
geom = file.read_geometry()
file.plot.geometry(axes="xyz").show()

INFO	Task(Task-2) nodify.node.139995064852048:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995064456192:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995064751152:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995065521056:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994995484064:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994995478832:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995064456192:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995064751152:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995065521056:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994995484064:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994995478832:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

In [85]:
from tqdm import tqdm
import numpy as np
import sisl

found = False

paths_sus=[]
for path in tqdm(paths):
    file = sisl.get_sile(path / "aiida.fdf")
    geom = file.read_geometry()

    dec = 1  # Decimal precision

    x_coords = np.round(geom.xyz[:, 0], dec)
    y_coords = np.round(geom.xyz[:, 1], dec)
    z_coords = np.round(geom.xyz[:, 2], dec)

    # Find atoms in minimum x-plane
    x_min = min(x_coords)

    for i, x in enumerate(x_coords):
        if x == x_min:
            for j, x1 in enumerate(x_coords):
                if x1 > x_min:
                    if z_coords[j] == z_coords[i] and y_coords[j] == y_coords[i]:
                        found = True

                        # print(path)
                        # print("x=", x_coords)
                        # print("y=", y_coords)
                        # print("z=", z_coords)

                        first_coords = (x_coords[i], y_coords[i], z_coords[i])
                        second_coords = (x_coords[j], y_coords[j], z_coords[j])
                        # print(f"\nFound atom {j} in second layer (X direction) beside atom {i} in first layer.")
                        # # print("Coords of the first layer atom:", first_coords)
                        # # print("Coords of the second layer atom:", second_coords)
                        # file.plot.geometry(axes="xyz").show()
                        paths_sus.append(path)

if not found:
    print("\nNo atoms stacked along X at same ZY position in ANY path.")
else:
    print("Yes.")

100%|██████████| 399/399 [00:06<00:00, 58.08it/s]

Yes.


In [86]:
i = 20
path = paths_sus[i]

file = sisl.get_sile(path / "aiida.fdf")
geom = file.read_geometry()
file.plot.geometry(axes="xyz").show()

INFO	Task(Task-2) nodify.node.139995075679232:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995083262288:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995094218800:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995092345504:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995092515584:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995094219424:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139995083262288:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995094218800:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995092345504:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995092515584:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139995094219424:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

In [ ]:
import sisl
import numpy as np

for path in paths_sus:
    file = sisl.get_sile(path / "aiida.fdf")
    geom = file.read_geometry()

    # Define a reasonable bonding cutoff (in Angstroms)
    bond_cutoff = 2.0

    # Loop over all atoms
    for i, atom_i in enumerate(geom):
        neighbors = geom.close(i, R=bond_cutoff)
        for j in neighbors:
            if geom.atoms.Z[i] == geom.atoms.Z[j]:
                print(f"⚠️ Bond detected between atoms {i} and {j} of type {geom[i].symbol}")
                print(f"Distance: {np.linalg.norm(geom.xyz[i] - geom.xyz[j]):.2f} Å")

IndexError: index 418 is out of bounds for axis 0 with size 8

In [112]:
bond_cutoff = 3.0  # Adjust based on expected bond lengths

for path in paths_sus:
    removed = False
    file = sisl.get_sile(path / "aiida.fdf")
    geom = file.read_geometry()
    for i in range(len(geom)):
        neighbors = geom.close(i, R=bond_cutoff)

        for j in neighbors:
            # Convert supercell index to unit-cell index
            idx_j = geom.asc2uc(j)

            # Skip self-comparison and check atomic type
            if i != idx_j and geom.atoms.Z[i] == geom.atoms.Z[idx_j]:
                dist = np.linalg.norm(geom.axyz(i) - geom.axyz(j))
                print(f"⚠️ Bond between same-type atoms {i} and {idx_j} at distance {dist:.2f} Å")
                if not removed:
                    paths_sus.remove(path)
                    removed = True


⚠️ Bond between same-type atoms 0 and 2 at distance 2.89 Å
⚠️ Bond between same-type atoms 0 and 2 at distance 1.71 Å
⚠️ Bond between same-type atoms 1 and 3 at distance 1.44 Å
⚠️ Bond between same-type atoms 1 and 3 at distance 2.94 Å
⚠️ Bond between same-type atoms 1 and 3 at distance 2.91 Å
⚠️ Bond between same-type atoms 2 and 0 at distance 2.89 Å
⚠️ Bond between same-type atoms 2 and 0 at distance 1.71 Å
⚠️ Bond between same-type atoms 3 and 1 at distance 1.44 Å
⚠️ Bond between same-type atoms 3 and 1 at distance 2.91 Å
⚠️ Bond between same-type atoms 3 and 1 at distance 2.94 Å
⚠️ Bond between same-type atoms 4 and 6 at distance 1.45 Å
⚠️ Bond between same-type atoms 4 and 6 at distance 2.93 Å
⚠️ Bond between same-type atoms 4 and 6 at distance 2.94 Å
⚠️ Bond between same-type atoms 5 and 7 at distance 1.71 Å
⚠️ Bond between same-type atoms 5 and 7 at distance 2.89 Å
⚠️ Bond between same-type atoms 6 and 4 at distance 1.45 Å
⚠️ Bond between same-type atoms 6 and 4 at distance 2.94

In [113]:
len(paths_sus)

27

In [115]:
# Plot all the structures
for k, path in enumerate(paths_sus):
    if k >= 15 and k < 30:
        file = sisl.get_sile(path / "aiida.fdf")
        geom = file.read_geometry()
        file.plot.geometry(axes="xyz").show()
    else:
        continue

INFO	Task(Task-2) nodify.node.139994963530656:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963396944:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963394784:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963404624:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963407936:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994965342976:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963396944:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963394784:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963404624:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963407936:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994965342976:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994963668544:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963397712:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994970662352:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963666432:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994965334000:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963665568:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963397712:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994970662352:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963666432:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994965334000:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963665568:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994963934720:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963926704:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963921328:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963919840:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963801008:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994971387712:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963926704:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963921328:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963919840:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963801008:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994971387712:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994964188272:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964176736:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963919072:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963929008:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963922576:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963924016:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964176736:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963919072:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963929008:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963922576:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963924016:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994963799280:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964019536:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964169776:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994965115392:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964180144:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964169440:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964019536:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994964169776:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994965115392:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994964180144:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994964169440:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

...

INFO	Task(Task-2) nodify.node.139994963000688:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962991616:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962985040:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962992336:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962985664:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962987920:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962991616:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962985040:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962992336:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962985664:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962987920:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994963225408:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962986864:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962795488:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963218976:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962334912:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962341728:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962986864:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962795488:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963218976:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962334912:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962341728:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994961318240:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962337312:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962737440:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961345424:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963162848:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962524832:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994962337312:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962737440:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994961345424:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963162848:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994962524832:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994961662928:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963219840:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963660432:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961641984:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994965290800:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994964675040:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963219840:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963660432:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994961641984:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994965290800:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994964675040:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

INFO	Task(Task-2) nodify.node.139994961815088:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961805728:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961345280:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961808896:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994963160736:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994965290608:node.py:get()- Evaluated because inputs changed.
INFO	Task(Task-2) nodify.node.139994961805728:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994961345280:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994961808896:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994963160736:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.node.139994965290608:node.py:get()- No need to evaluate
INFO	Task(Task-2) nodify.

In [104]:
print(geom.atoms)
print(geom.atoms.Z)

Atoms{species: 2,
 Atom{B, Z: 5, mass(au): 10.81000, maxR: 3.73962,
  AtomicOrbital{2sZ1, q0: 2.0, SphericalOrbital{l: 0, R: 3.0242091780742824, q0: 2.0}},
  AtomicOrbital{2sZ2, q0: 0.0, SphericalOrbital{l: 0, R: 2.0234137240957186, q0: 0.0}},
  AtomicOrbital{2pyZ1, q0: 0.3333333333333333, SphericalOrbital{l: 1, R: 3.739619420300641, q0: 1.0}},
  AtomicOrbital{2pzZ1, q0: 0.3333333333333333, SphericalOrbital{l: 1, R: 3.739619420300641, q0: 1.0}},
  AtomicOrbital{2pxZ1, q0: 0.3333333333333333, SphericalOrbital{l: 1, R: 3.739619420300641, q0: 1.0}},
  AtomicOrbital{2pyZ2, q0: 0.0, SphericalOrbital{l: 1, R: 2.5125394455189993, q0: 0.0}},
  AtomicOrbital{2pzZ2, q0: 0.0, SphericalOrbital{l: 1, R: 2.5125394455189993, q0: 0.0}},
  AtomicOrbital{2pxZ2, q0: 0.0, SphericalOrbital{l: 1, R: 2.5125394455189993, q0: 0.0}},
  AtomicOrbital{3dxyZ1P, q0: 0.0, SphericalOrbital{l: 2, R: 3.739619420300641, q0: 0.0}},
  AtomicOrbital{3dyzZ1P, q0: 0.0, SphericalOrbital{l: 2, R: 3.739619420300641, q0: 0.0}},
